In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Audio File Name Extraction**

In [14]:
NameList = []
import os
filepath = '/content/drive/My Drive/EvueMe New Dataset/50 Can_be_Considered Candidates/Audio (50 Can_be_Considered)'
for audio in os.listdir(filepath):
  NameList.append(audio.split('.wav')[0])

NameList, len(NameList), type(NameList)

(['ABHISHEK SINGH2',
  'ANJALI BHATT',
  'ARUSH SHARMA',
  'Aavriti Arora',
  'Aayush Upadhyay',
  'Abhishek Singh1',
  'Abhishek Singh3',
  'Aliza Nadir',
  'Amit Kumar Singh',
  'Amrita Singh',
  'Anish Jindal',
  'Anubhav Jain',
  'Arshdeep kaur',
  'Athulya Krishnan',
  'Ayush Rawat',
  'Bharvi Shaht',
  'Debajit Kumar Bhatta',
  'Dhairya Nagdev',
  'Divya Sunil Nair',
  'Gagandeep Singh',
  'Hariharan K S',
  'Harshitha M',
  'Jayasuriya Nandhagopal',
  'Jaykumar Maniyar',
  'Lakshay kapoor',
  'Lavendra Singh',
  'Nayana Rose C W',
  'Neha Chauhan',
  'Nikhil Shukla',
  'Pawan Negi',
  'Prabhat Mishra',
  'Pratyusha',
  'Ravi Kumar Singh',
  'Ravisha Khurana',
  'Reshabh S P',
  'Riya Sanjay Gangwal',
  'Ronit Bainipal',
  'SANJAY GM',
  'SHANTANU MANISH',
  'Samarth Rajesh Shirodkar',
  'Shambhavi Singhr',
  'Shanmukha sri manikanta ravuri',
  'Shivangi Bakshi',
  'Shivangi Chaturvedi',
  'Shivani Sharma',
  'Shivani sejra',
  'Suyash Goenka',
  'Vansh Yadav',
  'Vishal Bhati',


# **PRAAT through Parselmouth**

In [7]:
import glob
import numpy as np
import pandas as pd
import os
import librosa

!pip install praat-parselmouth
!pip install -U praat-parselmouth
import parselmouth

from urllib.parse import quote
from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

     |████████████████████████████████| 9.0MB 2.5MB/s 
Requirement already up-to-date: praat-parselmouth in /usr/local/lib/python3.6/dist-packages (0.3.3)


In [8]:
# This is the function to measure sound acoustics using default male parameters.

def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    F0_max = call(pitch, "Get maximum", 0, 0, unit, 'parabolic') # get maximum pitch
    F0_min = call(pitch, "Get minimum", 0, 0, unit, 'parabolic') # get minimum pitch
    F0_range = F0_max - F0_min  #get pitch range
    F0_mean = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    F0_std = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    F0_var = F0_std * F0_std  #get variance 
    F0_q25 = call(pitch, "Get quantile", 0,0, 0.25, unit)
    F0_q50 = call(pitch, "Get quantile", 0,0, 0.50, unit)
    F0_q75 = call(pitch, "Get quantile", 0,0, 0.75, unit)
    F0_IQR = F0_q75 - F0_q25
    
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, F0_min, F0_max, F0_range, F0_mean, F0_var, F0_std, F0_q25, F0_q50, F0_q75, F0_IQR, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [9]:
#This is the function to measure Intensity metrics
def measureIntensity(Sound):

  intensity = Sound.to_intensity()
  intensity = intensity.as_array()
  
  intensity_min = np.amin(intensity)
  intensity_max = np.amax(intensity)
  intensity_range = intensity_max - intensity_min
  intensity_mean = np.mean(intensity)
  intensity_median = np.median(intensity)
  intensity_var = np.var(intensity)
  intensity_std = np.std(intensity)
  intensity_q25 = np.percentile(intensity, 25)
  intensity_q50 = np.percentile(intensity, 50)
  intensity_q75 = np.percentile(intensity, 75)
  intensity_IQR = intensity_q75 - intensity_q25

  return intensity_min, intensity_max, intensity_range, intensity_mean, intensity_median, intensity_var, intensity_std, intensity_q25, intensity_q50, intensity_q75, intensity_IQR
  

In [10]:
# This function measures formants using Formant Position formula
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']

    import numpy as np
    f1_list = np.array(f1_list)
    f2_list = np.array(f2_list)
    f3_list = np.array(f3_list)
    f4_list = np.array(f4_list)

    # calculate min formants across pulses
    f1_min = np.amin(f1_list)  
    f2_min = np.amin(f2_list)  
    f3_min = np.amin(f3_list) 
    f4_min = np.amin(f4_list)

    # calculate max formants across pulses
    f1_max = np.amax(f1_list)  
    f2_max = np.amax(f2_list)  
    f3_max = np.amax(f3_list) 
    f4_max = np.amax(f4_list)

    # calculate range formants across pulses
    f1_range = f1_max - f1_min
    f2_range = f2_max - f2_min  
    f3_range = f3_max - f3_min 
    f4_range = f4_max - f4_min
    
    # calculate mean formants across pulses
    f1_mean = np.mean(f1_list)
    f2_mean = np.mean(f2_list)
    f3_mean = np.mean(f3_list)
    f4_mean = np.mean(f4_list)
    
    # calculate median formants across pulses
    f1_median = np.median(f1_list)
    f2_median = np.median(f2_list)
    f3_median = np.median(f3_list)
    f4_median = np.median(f4_list)

    # calculate variance formants across pulses
    f1_var = np.var(f1_list)
    f2_var = np.var(f2_list)
    f3_var = np.var(f3_list)
    f4_var = np.var(f4_list)

    # calculate standard deviation formants across pulses
    f1_std_dev = np.std(f1_list)
    f2_std_dev = np.std(f2_list)
    f3_std_dev = np.std(f3_list)
    f4_std_dev = np.std(f4_list)

    f1_q25 = np.percentile(f1_list, 25)    #First Quartile
    f1_q50 = np.percentile(f1_list, 50)    #Second Quartile
    f1_q75 = np.percentile(f1_list, 75)    #Third Quartile
    f1_IQR = f1_q75 - f1_q25
    f2_q25 = np.percentile(f2_list, 25)
    f2_q50 = np.percentile(f2_list, 50)
    f2_q75 = np.percentile(f2_list, 75)
    f2_IQR = f2_q75 - f2_q25
    f3_q25 = np.percentile(f3_list, 25)
    f3_q50 = np.percentile(f3_list, 50)
    f3_q75 = np.percentile(f3_list, 75)
    f3_IQR = f3_q75 - f3_q25
    f4_q25 = np.percentile(f4_list, 25)
    f4_q50 = np.percentile(f4_list, 50)
    f4_q75 = np.percentile(f4_list, 75)
    f4_IQR = f4_q75 - f4_q25
  
    return f1_min, f1_max, f1_range, f1_mean, f1_median, f1_var, f1_std_dev, f1_q25, f1_q50, f1_q75, f1_IQR, f2_min, f2_max, f2_range, f2_mean, f2_median, f2_var, f2_std_dev, f2_q25, f2_q50, f2_q75, f2_IQR, f3_min, f3_max, f3_range, f3_mean, f3_median, f3_var, f3_std_dev, f3_q25, f3_q50, f3_q75, f3_IQR, f4_min, f4_max, f4_range, f4_mean, f4_median, f4_var, f4_std_dev, f4_q25, f4_q50, f4_q75, f4_IQR

In [11]:
#This is the function to measure pitch and magnitude apart from sound metrics derived from parselmouth 
def measureSound_Pitch_Magnitude(sound, wave_file):
    
    #calculate sound measures
    sound = sound.as_array()
    sound_min = np.amin(sound)
    sound_max = np.amin(sound)
    sound_range = sound_max - sound_min
    sound_mean = np.mean(sound)
    sound_median = np.median(sound)
    sound_var = np.var(sound)
    sound_std = np.std(sound)
    sound_q25 = np.percentile(sound, 25)
    sound_q50 = np.percentile(sound, 50)
    sound_q75 = np.percentile(sound, 75)
    sound_IQR = sound_q75 - sound_q25

    #calculate pitch & magnitude using librosa
    y , sr = librosa.load(wave_file, sr = 44100, mono=True)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_min = np.amin(pitches)
    pitch_max = np.amax(pitches)
    pitch_range = pitch_max - pitch_min
    pitch_mean = np.mean(pitches)
    pitch_median = np.median(pitches)
    pitch_var = np.var(pitches)
    pitch_std = np.std(pitches)
    pitch_q25 = np.percentile(pitches, 25)
    pitch_q50 = np.percentile(pitches, 50)
    pitch_q75 = np.percentile(pitches, 75)
    pitch_IQR = pitch_q75 - pitch_q25

    magnitude_min = np.amin(magnitudes)
    magnitude_max = np.amax(magnitudes)
    magnitude_range = magnitude_max - magnitude_min
    magnitude_mean = np.mean(magnitudes)
    magnitude_median = np.median(magnitudes)
    magnitude_var = np.var(magnitudes)
    magnitude_std = np.std(magnitudes)
    magnitude_q25 = np.percentile(magnitudes, 25)
    magnitude_q50 = np.percentile(magnitudes, 50)
    magnitude_q75 = np.percentile(magnitudes, 75)
    magnitude_IQR = magnitude_q75 - magnitude_q25

    return sound_min, sound_max, sound_range, sound_mean, sound_median, sound_var, sound_std, sound_q25, sound_q50, sound_q75, sound_IQR, pitch_min, pitch_max, pitch_range, pitch_mean, pitch_median, pitch_var, pitch_std, pitch_q25, pitch_q50, pitch_q75, pitch_IQR, magnitude_min, magnitude_max, magnitude_range, magnitude_mean, magnitude_median, magnitude_var, magnitude_std, magnitude_q25, magnitude_q50, magnitude_q75, magnitude_IQR

In [12]:
def runPCA(df):
    # z-score the Jitter and Shimmer measurements
    measures = ['localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 'apq11Shimmer', 'ddaShimmer']
    x = df.loc[:, measures].values
    x = StandardScaler().fit_transform(x)
    # PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['JitterPCA', 'ShimmerPCA'])
    principalDf
    return principalDf

In [13]:
# create lists to put the results
file_list = []
duration_list = []
F0_min_list = []
F0_max_list = []
F0_range_list = []
F0_mean_list = []
F0_var_list = []
F0_std_list = []
F0_q25_list = []
F0_q50_list = []
F0_q75_list = []
F0_IQR_list = []
hnr_list = []
localJitter_list = []
localabsoluteJitter_list = []
rapJitter_list = []
ppq5Jitter_list = []
ddpJitter_list = []
localShimmer_list = []
localdbShimmer_list = []
apq3Shimmer_list = []
aqpq5Shimmer_list = []
apq11Shimmer_list = []
ddaShimmer_list = []

intensity_min_list = []
intensity_max_list = []
intensity_range_list = []
intensity_mean_list = []
intensity_median_list = []
intensity_var_list = []
intensity_std_list = []
intensity_q25_list = []
intensity_q50_list = []
intensity_q75_list = []
intensity_IQR_list = []

f1_min_list = []
f2_min_list = []
f3_min_list = []
f4_min_list = []
f1_max_list = []
f2_max_list = []
f3_max_list = []
f4_max_list = []
f1_range_list = []
f2_range_list = []
f3_range_list = []
f4_range_list = []
f1_mean_list = []
f2_mean_list = []
f3_mean_list = []
f4_mean_list = []
f1_median_list = []
f2_median_list = []
f3_median_list = []
f4_median_list = []
f1_var_list = []
f2_var_list = []
f3_var_list = []
f4_var_list = []
f1_std_dev_list = []
f2_std_dev_list = []
f3_std_dev_list = []
f4_std_dev_list = []
f1_q25_list = []
f1_q50_list = []
f1_q75_list = []
f1_IQR_list = []
f2_q25_list = []
f2_q50_list = []
f2_q75_list = []
f2_IQR_list = []
f3_q25_list = []
f3_q50_list = []
f3_q75_list = []
f3_IQR_list = []
f4_q25_list = []
f4_q50_list = []
f4_q75_list = []
f4_IQR_list = []

sound_min_list = []
sound_max_list = []
sound_range_list = []
sound_mean_list = []
sound_median_list = []
sound_var_list = []
sound_std_list = []
sound_q25_list = []
sound_q50_list = []
sound_q75_list = []
sound_IQR_list = []

pitch_min_list = []
pitch_max_list = []
pitch_range_list = []
pitch_mean_list = []
pitch_median_list = []
pitch_var_list = []
pitch_std_list = []
pitch_q25_list = []
pitch_q50_list = []
pitch_q75_list = []
pitch_IQR_list = []

magnitude_min_list = []
magnitude_max_list = []
magnitude_range_list = []
magnitude_mean_list = []
magnitude_median_list = []
magnitude_var_list = []
magnitude_std_list = []
magnitude_q25_list = []
magnitude_q50_list = []
magnitude_q75_list = []
magnitude_IQR_list = []

# Go through all the wave files in the folder and measure all the acoustics
for wave_file in glob.glob("/content/drive/My Drive/EvueMe New Dataset/50 Can_be_Considered Candidates/Audio (50 Can_be_Considered)/*.wav"):

    sound = parselmouth.Sound(wave_file)

    (duration, F0_min, F0_max, F0_range, F0_mean, F0_var, F0_std, F0_q25, F0_q50, F0_q75, F0_IQR, hnr, localJitter, localabsoluteJitter, rapJitter, 
     ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, 
     apq11Shimmer, ddaShimmer) = measurePitch(sound, 75, 600, "Hertz")

    (intensity_min, intensity_max, intensity_range, intensity_mean, intensity_median, intensity_var, intensity_std, 
     intensity_q25, intensity_q50, intensity_q75, intensity_IQR) = measureIntensity(sound)

    (f1_min, f1_max, f1_range, f1_mean, f1_median, f1_var, f1_std_dev, f1_q25, f1_q50, f1_q75, f1_IQR, f2_min, f2_max, 
     f2_range, f2_mean, f2_median, f2_var, f2_std_dev, f2_q25, f2_q50, f2_q75, f2_IQR, f3_min, f3_max, f3_range, 
     f3_mean, f3_median, f3_var, f3_std_dev, f3_q25, f3_q50, f3_q75, f3_IQR, f4_min, f4_max, f4_range, f4_mean, 
     f4_median, f4_var, f4_std_dev, f4_q25, f4_q50, f4_q75, f4_IQR) = measureFormants(sound, wave_file, 75, 600)

    (sound_min, sound_max, sound_range, sound_mean, sound_median, sound_var, sound_std, sound_q25, sound_q50, 
     sound_q75, sound_IQR, pitch_min, pitch_max, pitch_range, pitch_mean, pitch_median, pitch_var, pitch_std, 
     pitch_q25, pitch_q50, pitch_q75, pitch_IQR, magnitude_min, magnitude_max, magnitude_range, magnitude_mean, 
     magnitude_median, magnitude_var, magnitude_std, magnitude_q25, magnitude_q50, 
     magnitude_q75, magnitude_IQR) = measureSound_Pitch_Magnitude(sound, wave_file)  

    duration_list.append(duration) 
    F0_min_list.append(F0_min) 
    F0_max_list.append(F0_max) 
    F0_range_list.append(F0_range)
    F0_mean_list.append(F0_mean)
    F0_var_list.append(F0_var)
    F0_std_list.append(F0_std)
    F0_q25_list.append(F0_q25)
    F0_q50_list.append(F0_q50)
    F0_q75_list.append(F0_q75)
    F0_IQR_list.append(F0_IQR)
    hnr_list.append(hnr) 
    
    # add raw jitter and shimmer measures
    localJitter_list.append(localJitter)
    localabsoluteJitter_list.append(localabsoluteJitter)
    rapJitter_list.append(rapJitter)
    ppq5Jitter_list.append(ppq5Jitter)
    ddpJitter_list.append(ddpJitter)
    localShimmer_list.append(localShimmer)
    localdbShimmer_list.append(localdbShimmer)
    apq3Shimmer_list.append(apq3Shimmer)
    aqpq5Shimmer_list.append(aqpq5Shimmer)
    apq11Shimmer_list.append(apq11Shimmer)
    ddaShimmer_list.append(ddaShimmer)

    # add the intensity measures
    intensity_min_list.append(intensity_min)
    intensity_max_list.append(intensity_max)
    intensity_range_list.append(intensity_range)
    intensity_mean_list.append(intensity_mean)
    intensity_median_list.append(intensity_median)
    intensity_var_list.append(intensity_var)
    intensity_std_list.append(intensity_std)
    intensity_q25_list.append(intensity_q25)
    intensity_q50_list.append(intensity_q50)
    intensity_q75_list.append(intensity_q75)
    intensity_IQR_list.append(intensity_IQR)
    
    # add the formant data
    f1_min_list.append(f1_min)
    f2_min_list.append(f2_min)
    f3_min_list.append(f3_min)
    f4_min_list.append(f4_min)

    f1_max_list.append(f1_max)
    f2_max_list.append(f2_max)
    f3_max_list.append(f3_max)
    f4_max_list.append(f4_max)

    f1_range_list.append(f1_range)
    f2_range_list.append(f2_range)
    f3_range_list.append(f3_range)
    f4_range_list.append(f4_range)

    f1_mean_list.append(f1_mean)
    f2_mean_list.append(f2_mean)
    f3_mean_list.append(f3_mean)
    f4_mean_list.append(f4_mean)

    f1_median_list.append(f1_median)
    f2_median_list.append(f2_median)
    f3_median_list.append(f3_median)
    f4_median_list.append(f4_median)

    f1_var_list.append(f1_var)
    f2_var_list.append(f2_var)
    f3_var_list.append(f3_var)
    f4_var_list.append(f4_var)

    f1_std_dev_list.append(f1_std_dev)
    f2_std_dev_list.append(f2_std_dev)
    f3_std_dev_list.append(f3_std_dev)
    f4_std_dev_list.append(f4_std_dev)
    
    f1_q25_list.append(f1_q25)
    f1_q50_list.append(f1_q50)
    f1_q75_list.append(f1_q75)
    f1_IQR_list.append(f1_IQR)
    f2_q25_list.append(f2_q25)
    f2_q50_list.append(f2_q50)
    f2_q75_list.append(f2_q75)
    f2_IQR_list.append(f2_IQR)
    f3_q25_list.append(f3_q25)
    f3_q50_list.append(f3_q50)
    f3_q75_list.append(f3_q75)
    f3_IQR_list.append(f3_IQR)
    f4_q25_list.append(f4_q25)
    f4_q50_list.append(f4_q50)
    f4_q75_list.append(f4_q75)
    f4_IQR_list.append(f4_IQR)

    #add the sound data
    sound_min_list.append(sound_min)
    sound_max_list.append(sound_max)
    sound_range_list.append(sound_range)
    sound_mean_list.append(sound_mean)
    sound_median_list.append(sound_median)
    sound_var_list.append(sound_var)
    sound_std_list.append(sound_std)
    sound_q25_list.append(sound_q25)
    sound_q50_list.append(sound_q50)
    sound_q75_list.append(sound_q75)
    sound_IQR_list.append(sound_IQR)

    #add the pitch data
    pitch_min_list.append(pitch_min)
    pitch_max_list.append(pitch_max)
    pitch_range_list.append(pitch_range)
    pitch_mean_list.append(pitch_mean)
    pitch_median_list.append(pitch_median)
    pitch_var_list.append(pitch_var)
    pitch_std_list.append(pitch_std)
    pitch_q25_list.append(pitch_q25)
    pitch_q50_list.append(pitch_q50)
    pitch_q75_list.append(pitch_q75)
    pitch_IQR_list.append(pitch_IQR)

    #add the magnitude data
    magnitude_min_list.append(magnitude_min)
    magnitude_max_list.append(magnitude_max)
    magnitude_range_list.append(magnitude_range)
    magnitude_mean_list.append(magnitude_mean)
    magnitude_median_list.append(magnitude_median)
    magnitude_var_list.append(magnitude_var)
    magnitude_std_list.append(magnitude_std)
    magnitude_q25_list.append(magnitude_q25)
    magnitude_q50_list.append(magnitude_q50)
    magnitude_q75_list.append(magnitude_q75)
    magnitude_IQR_list.append(magnitude_IQR)

In [14]:
# Add the data to Pandas
PRAAT_df = pd.DataFrame(np.column_stack([NameList, duration_list, F0_min_list, F0_max_list, F0_range_list, F0_mean_list, 
                                   F0_var_list, F0_std_list, F0_q25_list, F0_q50_list, F0_q75_list, F0_IQR_list, 
                                   hnr_list, localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, localdbShimmer_list, 
                                   apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list, 
                                   intensity_min_list, intensity_max_list, intensity_range_list, intensity_mean_list, 
                                   intensity_median_list, intensity_var_list, intensity_std_list, intensity_q25_list, 
                                   intensity_q50_list, intensity_q75_list, intensity_IQR_list, f1_min_list, f1_max_list, 
                                   f1_range_list, f1_mean_list, f1_median_list, f1_var_list, f1_std_dev_list, 
                                   f1_q25_list, f1_q50_list, f1_q75_list, f1_IQR_list, f2_min_list, f2_max_list, 
                                   f2_range_list, f2_mean_list, f2_median_list, f2_var_list, f2_std_dev_list, 
                                   f2_q25_list, f2_q50_list, f2_q75_list, f2_IQR_list, f3_min_list, f3_max_list, 
                                   f3_range_list, f3_mean_list, f3_median_list, f3_var_list, f3_std_dev_list, 
                                   f3_q25_list, f3_q50_list, f3_q75_list, f3_IQR_list, f4_min_list, f4_max_list, 
                                   f4_range_list, f4_mean_list, f4_median_list, f4_var_list, f4_std_dev_list, 
                                   f4_q25_list, f4_q50_list, f4_q75_list, f4_IQR_list, sound_min_list, sound_max_list, 
                                   sound_range_list, sound_mean_list, sound_median_list, sound_var_list,
                                   sound_std_list, sound_q25_list, sound_q50_list, sound_q75_list, sound_IQR_list, 
                                   pitch_min_list, pitch_max_list, pitch_range_list, pitch_mean_list, 
                                   pitch_median_list, pitch_var_list, pitch_std_list, pitch_q25_list, pitch_q50_list, 
                                   pitch_q75_list, pitch_IQR_list, magnitude_min_list, magnitude_max_list, 
                                   magnitude_range_list, magnitude_mean_list, magnitude_median_list, 
                                   magnitude_var_list, magnitude_std_list, magnitude_q25_list, magnitude_q50_list, 
                                   magnitude_q75_list, magnitude_IQR_list]),
                                   columns=['Name', 'duration', 'F0_min', 'F0_max', 'F0_range', 'F0_mean', 'F0_var',
                                            'F0_std', 'F0_q25', 'F0_q50', 'F0_q75', 'F0_IQR', 'HNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 
                                            'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer',
                                            'apq11Shimmer', 'ddaShimmer', 'intensity_min', 'intensity_max',
                                            'intensity_range', 'intensity_mean', 'intensity_median', 'intensity_var', 
                                            'intensity_std', 'intensity_q25', 'intensity_q50', 'intensity_q75', 
                                            'intensity_IQR', 'f1_min','f1_max', 'f1_range', 'f1_mean', 'f1_median', 
                                            'f1_var','f1_std_dev', 'f1_q25', 'f1_q50', 'f1_q75', 'f1_IQR', 'f2_min', 
                                            'f2_max', 'f2_range', 'f2_mean', 'f2_median', 'f2_var','f2_std_dev', 
                                            'f2_q25', 'f2_q50', 'f2_q75', 'f2_IQR', 'f3_min', 'f3_max', 'f3_range', 
                                            'f3_mean', 'f3_median', 'f3_var', 'f3_std_dev', 'f3_q25', 'f3_q50', 
                                            'f3_q75', 'f3_IQR', 'f4_min','f4_max', 'f4_mean', 'f4_range', 'f4_median', 
                                            'f4_var', 'f4_std_dev', 'f4_q25', 'f4_q50', 'f4_q75', 'f4_IQR', 
                                            'sound_min', 'sound_max', 'sound_range', 'sound_mean', 'sound_median', 
                                            'sound_var', 'sound_std', 'sound_q25', 'sound_q50', 'sound_q75', 
                                            'sound_IQR', 'pitch_min', 'pitch_max', 'pitch_range', 'pitch_mean', 
                                            'pitch_median', 'pitch_var', 'pitch_std', 'pitch_q25', 'pitch_q50', 
                                            'pitch_q75', 'pitch_IQR', 'magnitude_min', 'magnitude_max', 
                                            'magnitude_range', 'magnitude_mean', 'magnitude_median', 'magnitude_var', 
                                            'magnitude_std', 'magnitude_q25', 'magnitude_q50', 'magnitude_q75', 
                                            'magnitude_IQR'])

pcaData = runPCA(PRAAT_df) # Run jitter and shimmer PCA
PRAAT_df = pd.concat([PRAAT_df, pcaData], axis=1) # Add PCA data
# reload the data so it's all numbers
PRAAT_df.to_csv("PRAAT_Features_1.csv", index=False)
PRAAT_df = pd.read_csv('PRAAT_Features_1.csv', header=0)

In [15]:
PRAAT_df['pF'] = (zscore(PRAAT_df.f1_median) + zscore(PRAAT_df.f2_median) + zscore(PRAAT_df.f3_median) + zscore(PRAAT_df.f4_median)) / 4
PRAAT_df['fdisp'] = (PRAAT_df['f4_median'] - PRAAT_df['f1_median']) / 3
PRAAT_df['avgFormant'] = (PRAAT_df['f1_median'] + PRAAT_df['f2_median'] + PRAAT_df['f3_median'] + PRAAT_df['f4_median']) / 4
PRAAT_df['mff'] = (PRAAT_df['f1_median'] * PRAAT_df['f2_median'] * PRAAT_df['f3_median'] * PRAAT_df['f4_median']) ** 0.25
# reload the data again
PRAAT_df.to_csv("PRAAT_Features_1.csv", index=False)
PRAAT_df = pd.read_csv('PRAAT_Features_1.csv', header=0)

PRAAT_df['fitch_vtl'] = ((1 * (35000 / (4 * PRAAT_df['f1_median']))) +
                   (3 * (35000 / (4 * PRAAT_df['f2_median']))) + 
                   (5 * (35000 / (4 * PRAAT_df['f3_median']))) + 
                   (7 * (35000 / (4 * PRAAT_df['f4_median'])))) / 4

xysum = (0.5 * PRAAT_df['f1_median']) + (1.5 * PRAAT_df['f2_median']) + (2.5 * PRAAT_df['f3_median']) + (3.5 * PRAAT_df['f4_median'])
xsquaredsum = (0.5 ** 2) + (1.5 ** 2) + (2.5 ** 2) + (3.5 ** 2)
PRAAT_df['delta_f'] = xysum / xsquaredsum

PRAAT_df['vtl_delta_f'] = 35000 / (2 * PRAAT_df['delta_f'])
# Write out the final dataframe
PRAAT_df.to_csv("PRAAT_Features_2.csv", index=False)
PRAAT_df = pd.read_csv('PRAAT_Features_2.csv')
PRAAT_df

,Name,duration,F0_min,F0_max,F0_range,F0_mean,F0_var,F0_std,F0_q25,F0_q50,F0_q75,F0_IQR,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,intensity_min,intensity_max,intensity_range,intensity_mean,intensity_median,intensity_var,intensity_std,intensity_q25,intensity_q50,intensity_q75,intensity_IQR,f1_min,f1_max,f1_range,f1_mean,f1_median,...,sound_range,sound_mean,sound_median,sound_var,sound_std,sound_q25,sound_q50,sound_q75,sound_IQR,pitch_min,pitch_max,pitch_range,pitch_mean,pitch_median,pitch_var,pitch_std,pitch_q25,pitch_q50,pitch_q75,pitch_IQR,magnitude_min,magnitude_max,magnitude_range,magnitude_mean,magnitude_median,magnitude_var,magnitude_std,magnitude_q25,magnitude_q50,magnitude_q75,magnitude_IQR,JitterPCA,ShimmerPCA,pF,fdisp,avgFormant,mff,fitch_vtl,delta_f,vtl_delta_f
0,ABHISHEK SINGH2,109.505306,68.112569,593.092309,524.979740,152.686446,7187.506489,84.779163,119.120220,126.337151,134.956576,15.836356,13.007344,0.025276,0.000169,0.013191,0.014607,0.039572,0.160936,1.438727,0.070076,0.095226,0.144646,0.210229,19.475947,68.423025,48.947078,38.510927,38.292931,174.542588,13.211457,24.975232,38.292931,50.756221,25.780989,72.545579,1993.546608,1921.001029,435.707553,399.946408,...,0.0,-0.000009,-0.000031,0.000058,0.007636,-0.000946,-0.000031,0.000610,0.001556,0.0,3994.3870,3994.3870,28.666824,0.0,66199.5300,257.29270,0.0,0.0,0.0,0.0,0.0,36.688930,36.688930,0.004683,0.0,0.023674,0.153864,0.0,0.0,0.0,0.0,2.427142,-0.181176,1.214117,1204.200576,2237.866620,1679.142012,16.911228,1131.693253,15.463554
1,ANJALI BHATT,87.399909,75.287073,593.087570,517.800497,216.886091,3855.422830,62.092051,192.921553,205.198835,220.859763,27.938210,13.843038,0.023840,0.000111,0.012144,0.014417,0.036432,0.128912,1.300226,0.050414,0.084408,0.158058,0.151243,-300.000000,87.942389,387.942389,50.610233,56.829671,1632.052155,40.398665,30.275284,56.829671,75.019051,44.743767,75.616178,2624.468474,2548.852296,429.456893,382.865351,...,0.0,-0.000064,-0.000031,0.014490,0.120373,-0.008026,-0.000031,0.003387,0.011414,0.0,3993.8872,3993.8872,13.344493,0.0,23568.6720,153.52092,0.0,0.0,0.0,0.0,0.0,354.518520,354.518520,0.076855,0.0,6.270486,2.504094,0.0,0.0,0.0,0.0,0.363718,0.111852,0.044446,1002.274462,1996.260592,1528.899728,18.491493,997.890792,17.536989
2,ARUSH SHARMA,193.236463,67.537765,497.523194,429.985429,173.984366,960.642356,30.994231,156.437229,168.941669,185.217165,28.779936,14.110146,0.021433,0.000123,0.009819,0.012335,0.029457,0.106018,1.083293,0.039489,0.061960,0.117341,0.118468,-300.000000,87.270828,387.270828,61.186026,71.201267,731.826518,27.052292,50.544892,71.201267,78.422329,27.877437,68.914271,2553.516945,2484.602674,415.479569,350.550500,...,0.0,-0.000035,-0.000031,0.018710,0.136785,-0.039093,-0.000031,0.025330,0.064423,0.0,3994.2917,3994.2917,13.828155,0.0,32584.1860,180.51090,0.0,0.0,0.0,0.0,0.0,317.334720,317.334720,0.110950,0.0,8.123682,2.850207,0.0,0.0,0.0,0.0,-2.086026,0.999134,-0.839011,948.155713,1846.469102,1404.336227,20.125922,928.639443,18.844774
3,Aavriti Arora,224.908481,79.479300,599.722329,520.243029,249.103472,4214.096322,64.916071,213.112403,230.272690,260.120393,47.007990,12.866697,0.020477,0.000083,0.010055,0.011530,0.030165,0.137781,1.304264,0.059200,0.083268,0.136434,0.177601,-300.000000,82.451050,382.451050,26.348463,58.130816,10700.433084,103.442898,47.069263,58.130816,66.578058,19.508795,69.956548,2407.833874,2337.877326,459.128445,441.633032,...,0.0,-0.000019,0.000000,0.001884,0.043401,-0.008331,0.000000,0.007599,0.015930,0.0,3994.2470,3994.2470,19.996810,0.0,43755.5820,209.17834,0.0,0.0,0.0,0.0,0.0,174.570020,174.570020,0.031727,0.0,0.739338,0.859848,0.0,0.0,0.0,0.0,-0.357480,-1.050413,0.369430,1063.777427,2036.685864,1590.664586,17.665587,1022.605605,17.113147
4,Aayush Upadhyay,141.757823,74.473179,587.835658,513.362479,143.220480,1874.694001,43.297737,128.518839,135.311127,143.715367,15.196527,12.695521,0.019135,

In [ ]:
PRAAT_df1 = pd.read_csv('/content/gdrive/My Drive/EvueMe New Dataset/Praat Features/PRAAT_Features(1-10).csv')
PRAAT_df2 = pd.read_csv('/content/gdrive/My Drive/EvueMe New Dataset/Praat Features/PRAAT_Features(11-A).csv')
PRAAT_df3 = pd.read_csv('/content/gdrive/My Drive/EvueMe New Dataset/Praat Features/PRAAT_Features(B-G).csv')
PRAAT_df4 = pd.read_csv('/content/gdrive/My Drive/EvueMe New Dataset/Praat Features/PRAAT_Features(H-M).csv')
PRAAT_df5 = pd.read_csv('/content/gdrive/My Drive/EvueMe New Dataset/Praat Features/PRAAT_Features(N-R).csv')
PRAAT_df6 = pd.read_csv('/content/gdrive/My Drive/EvueMe New Dataset/Praat Features/PRAAT_Features(S-Z).csv')

PRAAT_df = pd.concat([PRAAT_df1, PRAAT_df2, PRAAT_df3, PRAAT_df4, PRAAT_df5, PRAAT_df6], ignore_index = False)
PRAAT_df

,Name,duration,F0_min,F0_max,F0_range,F0_mean,F0_var,F0_std,F0_q25,F0_q50,F0_q75,F0_IQR,HNR,localJitter,localabsoluteJitter,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,intensity_min,intensity_max,intensity_range,intensity_mean,intensity_median,intensity_var,intensity_std,intensity_q25,intensity_q50,intensity_q75,intensity_IQR,f1_min,f1_max,f1_range,f1_mean,f1_median,...,sound_range,sound_mean,sound_median,sound_var,sound_std,sound_q25,sound_q50,sound_q75,sound_IQR,pitch_min,pitch_max,pitch_range,pitch_mean,pitch_median,pitch_var,pitch_std,pitch_q25,pitch_q50,pitch_q75,pitch_IQR,magnitude_min,magnitude_max,magnitude_range,magnitude_mean,magnitude_median,magnitude_var,magnitude_std,magnitude_q25,magnitude_q50,magnitude_q75,magnitude_IQR,JitterPCA,ShimmerPCA,pF,fdisp,avgFormant,mff,fitch_vtl,delta_f,vtl_delta_f
0,Aarthi S R,175.310658,68.936566,592.699852,523.763286,194.014235,1999.042010,44.710648,169.969718,185.606176,206.483287,36.513569,13.428431,0.022372,0.000116,0.011444,0.013236,0.034331,0.135871,1.315683,0.060371,0.083746,0.144936,0.181114,-300.000000,87.102170,387.102170,67.715830,73.076275,427.690405,20.680677,58.719954,73.076275,79.037655,20.317701,70.559284,2115.705995,2045.146712,450.951887,425.260402,...,0.0,-0.000063,0.000092,0.021063,0.145130,-0.042969,0.000092,0.045776,0.088745,0.0,3994.3176,3994.3176,6.678018,0.0,11597.328,107.690895,0.0,0.0,0.0,0.0,0.0,310.06160,310.06160,0.116585,0.0,9.189037,3.031343,0.0,0.0,0.0,0.0,2.672499,-0.566760,0.748550,1109.894410,2041.294375,1553.605749,18.099002,1042.474403,16.786983
1,Aakansha Das,57.051429,71.145507,569.751473,498.605966,229.100418,2651.187631,51.489685,211.429363,223.296819,240.230560,28.801198,16.358071,0.015282,0.000067,0.007409,0.008117,0.022226,0.090198,0.968836,0.035449,0.051807,0.089033,0.106346,-300.000000,86.787554,386.787554,46.271584,65.477401,3340.788938,57.799558,24.801346,65.477401,77.504739,52.703393,106.126914,2126.752871,2020.625957,513.930879,488.299010,...,0.0,-0.000246,-0.000092,0.015341,0.123860,-0.010345,-0.000092,0.013885,0.024231,0.0,3994.2783,3994.2783,5.360908,0.0,10439.351,102.173140,0.0,0.0,0.0,0.0,0.0,305.79245,305.79245,0.091477,0.0,6.763953,2.600760,0.0,0.0,0.0,0.0,-2.554543,-0.449516,0.695574,991.804142,1984.402656,1583.315060,17.691022,992.564362,17.631098
2,Aakash Bomb,85.658413,103.139328,585.449977,482.310650,141.613613,1023.568473,31.993257,129.784956,137.808091,146.323466,16.538510,12.439118,0.011208,0.000079,0.004394,0.006027,0.013182,0.099627,0.975357,0.034916,0.057070,0.105127,0.104747,4.934689,84.718034,79.783345,60.216203,74.177527,690.854710,26.284115,46.233877,74.177527,79.477241,33.243363,81.746711,2126.938951,2045.192240,476.866226,444.296966,...,0.0,-0.000314,-0.000214,0.020057,0.141622,-0.029205,-0.000214,0.035034,0.064240,0.0,3994.2422,3994.2422,8.388567,0.0,19390.803,139.250870,0.0,0.0,0.0,0.0,0.0,225.57591,225.57591,0.132693,0.0,8.222990,2.867576,0.0,0.0,0.0,0.0,-3.299707,-1.926247,0.299343,934.807197,1923.913133,1522.601387,18.419070,958.509413,18.257515
3,Aaditya Dinesh Thokade,58.444626,88.100092,596.412313,508.312221,144.695397,819.423722,28.625578,133.692831,141.090067,151.757964,18.065132,16.000211,0.018209,0.000126,0.008954,0.010202,0.026862,0.087621,0.897439,0.030008,0.043323,0.091629,0.090024,-300.000000,84.433335,384.433335,0.311025,65.579262,17979.828207,134.088882,14.440378,65.579262,75.024152,60.583773,69.693999,2714.849833,2645.155834,377.399285,322.256865,...,0.0,-0.000022,0.000000,0.008925,0.094473,-0.010956,0.000000,0.017578,0.028534,0.0,3993.4760,3993.4760,13.102633,0.0,29946.023,173.049200,0.0,0.0,0.0,0.0,0.0,203.63159,203.63159,0.075740,0.0,3.929103,1.982197,0.0,0.0,0.0,0.0,-1.980788,1.204087,-0.625506,1058.068363,1878.628805,1379.596947,20.617909,963.679646,18.159562
4,Adarsh,34.597732,77.205355,531.336245,454.130890,140.104529,1921.542759,43.835405,124.640560,131.554730,139.712242,15.071682,16.751255,0.019935,0.000144,0.009135,0.

In [ ]:
PRAAT_df.to_csv("/content/gdrive/My Drive/EvueMe New Dataset/Praat Features/PRAAT_Features(A-Z).csv")

#**pyAudioAnalysis**

In [17]:
!git clone https://github.com/tyiannak/pyAudioAnalysis.git
!pip install -r pyAudioAnalysis/requirements.txt

Cloning into 'pyAudioAnalysis'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 2399 (delta 22), reused 10 (delta 5), pack-reused 2359
Receiving objects: 100% (2399/2399), 134.59 MiB | 27.95 MiB/s, done.
Resolving deltas: 100% (1475/1475), done.
     |████████████████████████████████| 13.1MB 179kB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 20.2MB 63.7MB/s 
     |████████████████████████████████| 153kB 48.1MB/s 
     |████████████████████████████████| 153kB 45.2MB/s 
     |████████████████████████████████| 6.7MB 39.9MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
  Created wheel for simplejson: filename=simplejson-3.16.0-cp36-cp36m-linux_x86_64.whl size=114015 sha256=713d3f236a6cdeaa6409c810c720ae8b064c92269bff9fc0be35e28148f514b7
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691

In [1]:
%cd pyAudioAnalysis

/content/pyAudioAnalysis


In [2]:
!pip install -e .

Obtaining file:///content/pyAudioAnalysis
  Running setup.py develop for pyAudioAnalysis


In [3]:
!pip uninstall python-magic
!apt-get install python3-magic

Uninstalling python-magic-0.4.18:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/magic.py
    /usr/local/lib/python3.6/dist-packages/python_magic-0.4.18.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled python-magic-0.4.18
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-mgc libmagic1 python3-magic
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 262 kB of archives.
After this operation, 5,257 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main am

**Single File at a time**

In [ ]:
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS
[Fs, x] = aIO.read_audio_file("/content/gdrive/My Drive/EvueMe New Dataset/Audio/pyAudioAnalysis not applicable/Aakash Bomb.wav")
segments = aS.silence_removal(x, Fs, 0.020, 0.020, smooth_window = 1.0, weight = 0.3, plot = True) #plot = True is optional & may not work for all audio files
segments

[[0.0, 0.86], [2.38, 5.3], [5.68, 37.38], [38.06, 72.28], [73.94, 74.64]]

**Multiple Audio Files at a time**

In [4]:
import glob
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS

segments_list = []
for wave_file in glob.glob("/content/drive/My Drive/EvueMe New Dataset/50 Can_be_Considered Candidates/Audio (50 Can_be_Considered)/*.wav"):
  [Fs, x] = aIO.read_audio_file(wave_file)
  segments = aS.silence_removal(x, Fs, 0.020, 0.020, smooth_window = 1.0, weight = 0.3)
  segments_list.append(segments)

segments_list, len(segments_list)

([[[0.0, 8.78],
   [9.8, 11.26],
   [12.5, 14.32],
   [14.52, 17.18],
   [17.28, 21.84],
   [22.64, 23.44],
   [23.900000000000002, 26.76],
   [27.5, 28.36],
   [29.16, 29.88],
   [30.76, 32.660000000000004],
   [32.94, 39.46],
   [39.96, 42.7],
   [44.34, 45.84],
   [47.02, 50.1],
   [51.22, 53.22],
   [53.86, 57.86],
   [58.76, 64.6],
   [66.06, 66.72],
   [68.08, 73.5],
   [74.46000000000001, 82.08],
   [82.54, 86.04],
   [87.0, 89.34],
   [90.04, 93.92],
   [95.12, 99.34]],
  [[0.16, 4.34],
   [5.68, 13.06],
   [13.36, 14.86],
   [15.38, 16.42],
   [16.94, 20.14],
   [20.62, 21.72],
   [22.02, 25.8],
   [25.98, 27.22],
   [27.44, 29.32],
   [30.32, 31.34],
   [32.04, 36.04],
   [36.46, 37.42],
   [38.800000000000004, 42.84],
   [43.04, 45.24],
   [48.06, 48.82],
   [49.34, 50.52],
   [50.78, 51.660000000000004],
   [54.22, 58.38],
   [59.04, 61.6],
   [61.68, 62.74],
   [65.14, 65.38],
   [65.54, 67.44],
   [70.4, 72.18],
   [72.60000000000001, 75.32000000000001]],
  [[2.62, 11.38]

In [5]:
Count_Pauses_list = []
for i in range(0,len(segments_list)):
  Count_Pauses_list.append(len(segments_list[i])-1)
Count_Pauses_list, len(Count_Pauses_list)

([23,
  23,
  29,
  11,
  31,
  9,
  8,
  24,
  16,
  20,
  9,
  11,
  24,
  11,
  13,
  5,
  7,
  4,
  2,
  2,
  6,
  15,
  49,
  19,
  4,
  41,
  6,
  16,
  20,
  4,
  4,
  34,
  13,
  13,
  3,
  4,
  9,
  5,
  27,
  2,
  9,
  29,
  34,
  3,
  17,
  5,
  18,
  3,
  19,
  49],
 50)

In [6]:
Pauses_list = [[] for i in range(len(Count_Pauses_list))]

for j in range(0,len(Count_Pauses_list)):
  for i in range(0, Count_Pauses_list[j]):
    Pauses_list[j].append(segments_list[j][i+1][0]-segments_list[j][i][1])

Pauses_list, len(Pauses_list)

([[1.0200000000000014,
   1.2400000000000002,
   0.1999999999999993,
   0.10000000000000142,
   0.8000000000000007,
   0.46000000000000085,
   0.7399999999999984,
   0.8000000000000007,
   0.8800000000000026,
   0.27999999999999403,
   0.5,
   1.6400000000000006,
   1.1799999999999997,
   1.1199999999999974,
   0.6400000000000006,
   0.8999999999999986,
   1.460000000000008,
   1.3599999999999994,
   0.960000000000008,
   0.46000000000000796,
   0.9599999999999937,
   0.7000000000000028,
   1.2000000000000028],
  [1.3399999999999999,
   0.29999999999999893,
   0.5200000000000014,
   0.5199999999999996,
   0.4800000000000004,
   0.3000000000000007,
   0.17999999999999972,
   0.22000000000000242,
   1.0,
   0.6999999999999993,
   0.4200000000000017,
   1.3800000000000026,
   0.19999999999999574,
   2.8200000000000003,
   0.5200000000000031,
   0.259999999999998,
   2.559999999999995,
   0.6599999999999966,
   0.0799999999999983,
   2.3999999999999986,
   0.1600000000000108,
   2.96000000

In [7]:
# Statistical Metrics related to Pause
import numpy  as np

Total_Pause_Time_list = []
Min_Pause_Time_list = []
Max_Pause_Time_list =[]
Range_Pause_Time_list = []
Mean_Pause_Time_list = []
Median_Pause_Time_list = []
Var_Pause_Time_list = []
StdDev_Pause_Time_list = []
Q25_Pause_Time_list =[]
Q50_Pause_Time_list =[]
Q75_Pause_Time_list = []
IQR_Pause_Time_list = []

# converting list to array 
for i in range(0,len(Pauses_list)):
  Pauses_list[i] = np.array(Pauses_list[i]) 
  Total_Pause_Time = np.sum(Pauses_list[i])
  Total_Pause_Time_list.append(Total_Pause_Time)
  if (len(Pauses_list[i]) != 0):
    Min_Pause_Time = np.amin(Pauses_list[i])
    Min_Pause_Time_list.append(Min_Pause_Time)
    Max_Pause_Time = np.amax(Pauses_list[i])
    Max_Pause_Time_list.append(Max_Pause_Time)
    Range_Pause_Time = Max_Pause_Time - Min_Pause_Time
    Range_Pause_Time_list.append(Range_Pause_Time)
    Mean_Pause_Time = np.mean(Pauses_list[i])
    Mean_Pause_Time_list.append(Mean_Pause_Time)
    Median_Pause_Time = np.median(Pauses_list[i])
    Median_Pause_Time_list.append(Median_Pause_Time)
    Var_Pause_Time = np.var(Pauses_list[i])
    Var_Pause_Time_list.append(Var_Pause_Time)
    StdDev_Pause_Time = np.std(Pauses_list[i])
    StdDev_Pause_Time_list.append(StdDev_Pause_Time)
    Q25_Pause_Time = np.percentile(Pauses_list[i],25)
    Q25_Pause_Time_list.append(Q25_Pause_Time)
    Q50_Pause_Time = np.percentile(Pauses_list[i],50)
    Q50_Pause_Time_list.append(Q50_Pause_Time)
    Q75_Pause_Time = np.percentile(Pauses_list[i],75)
    Q75_Pause_Time_list.append(Q75_Pause_Time)
    IQR_Pause_Time = Q75_Pause_Time - Q50_Pause_Time
    IQR_Pause_Time_list.append(IQR_Pause_Time)
  else:
    Min_Pause_Time_list.append(0)
    Max_Pause_Time_list.append(0)
    Range_Pause_Time_list.append(0)
    Mean_Pause_Time_list.append(0)
    Median_Pause_Time_list.append(0)
    Var_Pause_Time_list.append(0)
    StdDev_Pause_Time_list.append(0)
    Q25_Pause_Time_list.append(0)
    Q50_Pause_Time_list.append(0)
    Q75_Pause_Time_list.append(0)
    IQR_Pause_Time_list.append(0)

In [8]:
#Number of long pauses (>3 Sec)

Count_Long_Pauses_list = []

for j in range(0,len(Pauses_list)):
  Count_Long_Pauses = 0
  for i in Pauses_list[j]:
    if i > 3:
      Count_Long_Pauses = Count_Long_Pauses + 1
  Count_Long_Pauses_list.append(Count_Long_Pauses)
 
Count_Long_Pauses_list, len(Count_Long_Pauses_list)

([0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  2,
  0,
  0,
  0,
  3,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  1,
  1,
  0,
  0],
 50)

In [9]:
# Number of Speaking Turns (i.e, speaking segments > 2 sec)

Count_Speaking_Turns_list = []

for j in range(0, len(segments_list)):
  Count_Speaking_Turns = 0
  for i in segments_list[j]:
    if (i[1] - i[0] > 2):
      Count_Speaking_Turns = Count_Speaking_Turns + 1
  Count_Speaking_Turns_list.append(Count_Speaking_Turns)

Count_Speaking_Turns_list, len(Count_Speaking_Turns_list)

([15,
  10,
  20,
  11,
  23,
  7,
  7,
  9,
  9,
  16,
  7,
  9,
  12,
  11,
  12,
  4,
  5,
  3,
  3,
  3,
  5,
  2,
  29,
  16,
  3,
  33,
  6,
  13,
  17,
  4,
  4,
  18,
  12,
  11,
  4,
  4,
  9,
  4,
  13,
  3,
  5,
  22,
  21,
  3,
  14,
  4,
  17,
  3,
  17,
  28],
 50)

In [10]:
# Number of Long Speaking Turns (i.e, speaking segments > 7 sec)

Count_Long_Speaking_Turns_list = []

for j in range(0, len(segments_list)):
  Count_Long_Speaking_Turns = 0
  for i in segments_list[j]:
    if (i[1] - i[0] > 7):
      Count_Long_Speaking_Turns = Count_Long_Speaking_Turns + 1
  Count_Long_Speaking_Turns_list.append(Count_Long_Speaking_Turns)

Count_Long_Speaking_Turns_list, len(Count_Long_Speaking_Turns_list)

([2,
  1,
  9,
  8,
  1,
  4,
  4,
  0,
  4,
  6,
  6,
  2,
  1,
  5,
  5,
  2,
  5,
  3,
  1,
  2,
  2,
  0,
  4,
  5,
  2,
  10,
  5,
  9,
  3,
  2,
  3,
  3,
  5,
  6,
  2,
  3,
  4,
  3,
  4,
  2,
  3,
  4,
  2,
  1,
  9,
  0,
  9,
  2,
  11,
  0],
 50)

In [11]:
Speaking_Duration_list = [[] for i in range(0,len(segments_list))]

for j in range(0,len(segments_list)):
  for i in segments_list[j]:
    Speaking_Duration_list[j].append(i[1] - i[0])

Speaking_Duration_list, len(Speaking_Duration_list)

([[8.78,
   1.459999999999999,
   1.8200000000000003,
   2.66,
   4.559999999999999,
   0.8000000000000007,
   2.8599999999999994,
   0.8599999999999994,
   0.7199999999999989,
   1.9000000000000021,
   6.520000000000003,
   2.740000000000002,
   1.5,
   3.0799999999999983,
   2.0,
   4.0,
   5.839999999999996,
   0.6599999999999966,
   5.420000000000002,
   7.61999999999999,
   3.5,
   2.3400000000000034,
   3.8799999999999955,
   4.219999999999999],
  [4.18,
   7.380000000000001,
   1.5,
   1.040000000000001,
   3.1999999999999993,
   1.0999999999999979,
   3.780000000000001,
   1.2399999999999984,
   1.879999999999999,
   1.0199999999999996,
   4.0,
   0.9600000000000009,
   4.039999999999999,
   2.200000000000003,
   0.759999999999998,
   1.1799999999999997,
   0.8800000000000026,
   4.160000000000004,
   2.5600000000000023,
   1.0600000000000023,
   0.23999999999999488,
   1.8999999999999915,
   1.7800000000000011,
   2.719999999999999],
  [8.760000000000002,
   0.5399999999999991

In [12]:
#Total Speaking Duration
Total_Speaking_Duration_list = []
Min_Speaking_Duration_list = []
Max_Speaking_Duration_list = []
Range_Speaking_Duration_list = []
Mean_Speaking_Duration_list = []
Median_Speaking_Duration_list = []
Var_Speaking_Duration_list = []
StdDev_Speaking_Duration_list = []
Q25_Speaking_Duration_list = []
Q50_Speaking_Duration_list = []
Q75_Speaking_Duration_list = []
IQR_Speaking_Duration_list = []

for i in range(len(Speaking_Duration_list)):
  Total_Speaking_Duration = np.sum(Speaking_Duration_list[i]) 
  Total_Speaking_Duration_list.append(Total_Speaking_Duration)
  Min_Speaking_Duration = np.amin(Speaking_Duration_list[i])  
  Min_Speaking_Duration_list.append(Min_Speaking_Duration)
  Max_Speaking_Duration = np.amax(Speaking_Duration_list[i])
  Max_Speaking_Duration_list.append(Max_Speaking_Duration)
  Range_Speaking_Duration = Max_Speaking_Duration - Min_Speaking_Duration
  Range_Speaking_Duration_list.append(Range_Speaking_Duration)
  Mean_Speaking_Duration = np.mean(Speaking_Duration_list[i])
  Mean_Speaking_Duration_list.append(Mean_Speaking_Duration)
  Median_Speaking_Duration = np.median(Speaking_Duration_list[i])
  Median_Speaking_Duration_list.append(Median_Speaking_Duration)
  Var_Speaking_Duration = np.var(Speaking_Duration_list[i])
  Var_Speaking_Duration_list.append(Var_Speaking_Duration)
  StdDev_Speaking_Duration = np.std(Speaking_Duration_list[i])
  StdDev_Speaking_Duration_list.append(StdDev_Speaking_Duration)
  Q25_Speaking_Duration = np.percentile(Speaking_Duration_list[i],25)
  Q25_Speaking_Duration_list.append(Q25_Speaking_Duration)
  Q50_Speaking_Duration = np.percentile(Speaking_Duration_list[i],50)
  Q50_Speaking_Duration_list.append(Q50_Speaking_Duration)
  Q75_Speaking_Duration = np.percentile(Speaking_Duration_list[i],75)
  Q75_Speaking_Duration_list.append(Q75_Speaking_Duration)
  IQR_Speaking_Duration = Q75_Speaking_Duration - Q25_Speaking_Duration
  IQR_Speaking_Duration_list.append(IQR_Speaking_Duration)

In [15]:
import pandas as pd
 
pyAudioAnalysis_df = pd.DataFrame(list(zip(NameList, Count_Pauses_list, Total_Pause_Time_list, Min_Pause_Time_list, Max_Pause_Time_list, Range_Pause_Time_list, Mean_Pause_Time_list, Median_Pause_Time_list, Var_Pause_Time_list, StdDev_Pause_Time_list, Q25_Pause_Time_list, Q50_Pause_Time_list, Q75_Pause_Time_list, IQR_Pause_Time_list, Count_Long_Pauses_list, Count_Speaking_Turns_list, Count_Long_Speaking_Turns_list, Total_Speaking_Duration_list, Min_Speaking_Duration_list, Max_Speaking_Duration_list, Range_Speaking_Duration_list, Mean_Speaking_Duration_list, Median_Speaking_Duration_list, Var_Speaking_Duration_list, StdDev_Speaking_Duration_list, Q25_Speaking_Duration_list, Q50_Speaking_Duration_list, Q75_Speaking_Duration_list, IQR_Speaking_Duration_list)))

pyAudioAnalysis_df.columns =  ['Name', 'Count_Pauses', 'Total_Pause_Time', 'Min_Pause_Time', 'Max_Pause_Time', 'Range_Pause_Time', 'Mean_Pause_Time', 'Median_Pause_Time', 'Var_Pause_Time', 'StdDev_Pause_Time', 'Q25_Pause_Time', 'Q50_Pause_Time', 'Q75_Pause_Time', 'IQR_Pause_Time', 'Count_Long_Pauses', 'Count_Speaking_Turns', 'Count_Long_Speaking_Turns', 'Total_Speaking_Duration', 'Min_Speaking_Duration', 'Max_Speaking_Duration', 'Range_Speaking_Duration', 'Mean_Speaking_Duration', 'Median_Speaking_Duration', 'Var_Speaking_Duration', 'StdDev_Speaking_Duration', 'Q25_Speaking_Duration', 'Q50_Speaking_Duration', 'Q75_Speaking_Duration', 'IQR_Speaking_Duration'] 
#pyAudioAnalysis_df.index = NameList
pyAudioAnalysis_df

,Name,Count_Pauses,Total_Pause_Time,Min_Pause_Time,Max_Pause_Time,Range_Pause_Time,Mean_Pause_Time,Median_Pause_Time,Var_Pause_Time,StdDev_Pause_Time,Q25_Pause_Time,Q50_Pause_Time,Q75_Pause_Time,IQR_Pause_Time,Count_Long_Pauses,Count_Speaking_Turns,Count_Long_Speaking_Turns,Total_Speaking_Duration,Min_Speaking_Duration,Max_Speaking_Duration,Range_Speaking_Duration,Mean_Speaking_Duration,Median_Speaking_Duration,Var_Speaking_Duration,StdDev_Speaking_Duration,Q25_Speaking_Duration,Q50_Speaking_Duration,Q75_Speaking_Duration,IQR_Speaking_Duration
0,ABHISHEK SINGH2,23,19.60,0.10,1.64,1.54,0.852174,0.88,0.156060,0.395045,0.570,0.88,1.150,2.700000e-01,0,15,2,79.74,0.66,8.78,8.12,3.322500,2.80,4.728244,2.174453,1.740,2.80,4.305,2.565
1,ANJALI BHATT,23,20.40,0.08,2.96,2.88,0.886957,0.52,0.798004,0.893311,0.280,0.52,1.170,6.500000e-01,0,10,1,54.76,0.24,7.38,7.14,2.281667,1.83,2.592497,1.610123,1.055,1.83,3.345,2.290
2,ARUSH SHARMA,29,15.40,0.08,1.36,1.28,0.531034,0.50,0.130058,0.360635,0.280,0.50,0.640,1.400000e-01,0,20,9,154.76,0.54,13.24,12.70,5.158667,5.53,13.673945,3.697830,1.705,5.53,7.810,6.105
3,Aavriti Arora,11,18.20,0.92,4.28,3.36,1.654545,1.36,0.916298,0.957234,0.990,1.36,1.770,4.100000e-01,1,11,8,188.40,0.80,65.40,64.60,15.700000,12.80,261.855733,16.181957,6.405,12.80,15.675,9.270
4,Aayush Upadhyay,31,21.28,0.08,2.70,2.62,0.686452,0.58,0.390358,0.624787,0.230,0.58,0.820,2.400000e-01,0,23,1,105.70,0.54,9.62,9.08,3.303125,3.00,4.145478,2.036045,1.640,3.00,4.710,3.070
5,Abhishek Singh1,9,8.04,0.12,2.60,2.48,0.893333,0.72,0.463289,0.680653,0.420,0.72,1.000,2.800000e-01,0,7,4,53.10,0.26,10.36,10.10,5.310000,5.41,13.886660,3.726481,1.540,5.41,9.075,7.535
6,Abhishek Singh3,8,3.76,0.12,1.24,1.12,0.470000,0.43,0.120700,0.347419,0.185,0.43,0.570,1.400000e-01,0,7,4,110.22,1.00,32.92,31.92,12.246667,6.96,135.290044,11.631425,3.140,6.96,23.320,20.180
7,Aliza Nadir,24,17.40,0.08,2.40,2.32,0.725000,0.55,0.338108,0.581471,0.250,0.55,1.015,4.650000e-01,0,9,0,47.00,0.34,5.80,5.46,1.880000,1.34,2.192832,1.480821,0.940,1.34,2.220,1.280
8,Amit Kumar Singh,16,12.68,0.08,2.80,2.72,0.792500,0.50,0.563644,0.750762,0.285,0.50,1.085,5.850000e-01,0,9,4,63.88,0.56,8.70,8.14,3.757647,3.38,7.755947,2.784950,1.260,3.38,5.940,4.680
9,Amrita Singh,20,14.98,0.08,2.80,2.72,0.749000,0.59,0.459859,0.678129,0.250,0.59,0.925,3.350000e-01,0,16,6,122.66,0.22,17.70,17.48,5.840952,5.62,19.308513,4.394145,2.180,5.62,7.160,4.980


In [16]:
pyAudioAnalysis_df['Total_Silence_Speaking_Ratio'] = pyAudioAnalysis_df['Total_Pause_Time'] / pyAudioAnalysis_df['Total_Speaking_Duration']
pyAudioAnalysis_df['Mean_Silence_Speaking_Ratio'] = pyAudioAnalysis_df['Mean_Pause_Time'] / pyAudioAnalysis_df['Mean_Speaking_Duration']
pyAudioAnalysis_df['StdDev_Silence_Speaking_Ratio'] = pyAudioAnalysis_df['StdDev_Pause_Time'] / pyAudioAnalysis_df['StdDev_Speaking_Duration']

pyAudioAnalysis_df

,Name,Count_Pauses,Total_Pause_Time,Min_Pause_Time,Max_Pause_Time,Range_Pause_Time,Mean_Pause_Time,Median_Pause_Time,Var_Pause_Time,StdDev_Pause_Time,Q25_Pause_Time,Q50_Pause_Time,Q75_Pause_Time,IQR_Pause_Time,Count_Long_Pauses,Count_Speaking_Turns,Count_Long_Speaking_Turns,Total_Speaking_Duration,Min_Speaking_Duration,Max_Speaking_Duration,Range_Speaking_Duration,Mean_Speaking_Duration,Median_Speaking_Duration,Var_Speaking_Duration,StdDev_Speaking_Duration,Q25_Speaking_Duration,Q50_Speaking_Duration,Q75_Speaking_Duration,IQR_Speaking_Duration,Total_Silence_Speaking_Ratio,Mean_Silence_Speaking_Ratio,StdDev_Silence_Speaking_Ratio
0,ABHISHEK SINGH2,23,19.60,0.10,1.64,1.54,0.852174,0.88,0.156060,0.395045,0.570,0.88,1.150,2.700000e-01,0,15,2,79.74,0.66,8.78,8.12,3.322500,2.80,4.728244,2.174453,1.740,2.80,4.305,2.565,0.245799,0.256486,0.181676
1,ANJALI BHATT,23,20.40,0.08,2.96,2.88,0.886957,0.52,0.798004,0.893311,0.280,0.52,1.170,6.500000e-01,0,10,1,54.76,0.24,7.38,7.14,2.281667,1.83,2.592497,1.610123,1.055,1.83,3.345,2.290,0.372535,0.388732,0.554809
2,ARUSH SHARMA,29,15.40,0.08,1.36,1.28,0.531034,0.50,0.130058,0.360635,0.280,0.50,0.640,1.400000e-01,0,20,9,154.76,0.54,13.24,12.70,5.158667,5.53,13.673945,3.697830,1.705,5.53,7.810,6.105,0.099509,0.102940,0.097526
3,Aavriti Arora,11,18.20,0.92,4.28,3.36,1.654545,1.36,0.916298,0.957234,0.990,1.36,1.770,4.100000e-01,1,11,8,188.40,0.80,65.40,64.60,15.700000,12.80,261.855733,16.181957,6.405,12.80,15.675,9.270,0.096603,0.105385,0.059154
4,Aayush Upadhyay,31,21.28,0.08,2.70,2.62,0.686452,0.58,0.390358,0.624787,0.230,0.58,0.820,2.400000e-01,0,23,1,105.70,0.54,9.62,9.08,3.303125,3.00,4.145478,2.036045,1.640,3.00,4.710,3.070,0.201325,0.207819,0.306863
5,Abhishek Singh1,9,8.04,0.12,2.60,2.48,0.893333,0.72,0.463289,0.680653,0.420,0.72,1.000,2.800000e-01,0,7,4,53.10,0.26,10.36,10.10,5.310000,5.41,13.886660,3.726481,1.540,5.41,9.075,7.535,0.151412,0.168236,0.182653
6,Abhishek Singh3,8,3.76,0.12,1.24,1.12,0.470000,0.43,0.120700,0.347419,0.185,0.43,0.570,1.400000e-01,0,7,4,110.22,1.00,32.92,31.92,12.246667,6.96,135.290044,11.631425,3.140,6.96,23.320,20.180,0.034114,0.038378,0.029869
7,Aliza Nadir,24,17.40,0.08,2.40,2.32,0.725000,0.55,0.338108,0.581471,0.250,0.55,1.015,4.650000e-01,0,9,0,47.00,0.34,5.80,5.46,1.880000,1.34,2.192832,1.480821,0.940,1.34,2.220,1.280,0.370213,0.385638,0.392668
8,Amit Kumar Singh,16,12.68,0.08,2.80,2.72,0.792500,0.50,0.563644,0.750762,0.285,0.50,1.085,5.850000e-01,0,9,4,63.88,0.56,8.70,8.14,3.757647,3.38,7.755947,2.784950,1.260,3.38,5.940,4.680,0.198497,0.210903,0.269578
9,Amrita Singh,20,14.98,0.08,2.80,2.72,0.749000,0.59,0.459859,0.678129,0.250,0.59,0.925,3.350000e-01,0,16,6,122.66,0.22,17.70,17.48,5.840952,5.62,19.308513,4.394145,2.180,5.62,7.160,4.980,0.122126,0.128233,0.154326


In [ ]:
pyAudioAnalysis_df.to_csv("/content/gdrive/My Drive/EvueMe New Dataset/Audio Features/pyAudioAnalysis_SpeakingActivity_Features.csv")

## **No need**

In [ ]:
import IPython.display as ipd
ipd.Audio('/content/gdrive/My Drive/EvueMe New Dataset/Audio/Aaditya Dinesh Thokade.wav') # load a WAV file

In [ ]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import ShortTermFeatures
import matplotlib.pyplot as plt
[Fs, x] = audioBasicIO.read_audio_file("/content/gdrive/My Drive/EvueMe New Dataset/Audio/Aaditya Dinesh Thokade.wav")
F, f_names = ShortTermFeatures.feature_extraction(x, Fs, 0.050*Fs, 0.025*Fs)
print(F)
print(f_names)
plt.subplot(2,1,1); plt.plot(F[0,:]); plt.xlabel('Frame no'); plt.ylabel(f_names[0]) 
plt.subplot(2,1,2); plt.plot(F[1,:]); plt.xlabel('Frame no'); plt.ylabel(f_names[1]); plt.show()

In [ ]:
!python pyAudioAnalysis/audioAnalysis.py featureExtractionFile -i '/content/gdrive/My Drive/EvueMe New Dataset/Audio/Aaditya Dinesh Thokade.wav' -mw 1.0 -ms 1.0 -sw 0.050 -ss 0.050 -o '/content/Aaditya Dinesh Thokade.wav'

In [ ]:
# Load data
df = pd.read_csv('/content/Aaditya Dinesh Thokade.wav_st.csv')
print("\nTotal shape of dataframe", {df.shape})

# Change the column names 
df.columns =['zcr', 'energy', 'energy_entropy', 'spectral_centroid', 'spectral_spread', 'spectral_entropy', 'spectral_flux', 'spectral_rolloff', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'chroma_1', 'chroma_2', 'chroma_3', 'chroma_4', 'chroma_5', 'chroma_6', 'chroma_7', 'chroma_8', 'chroma_9', 'chroma_10', 'chroma_11', 'chroma_12', 'chroma_std', 'delta zcr', 'delta energy', 'delta energy_entropy', 'delta spectral_centroid', 'delta spectral_spread', 'delta spectral_entropy', 'delta spectral_flux', 'delta spectral_rolloff', 'delta mfcc_1', 'delta mfcc_2', 'delta mfcc_3', 'delta mfcc_4', 'delta mfcc_5', 'delta mfcc_6', 'delta mfcc_7', 'delta mfcc_8', 'delta mfcc_9', 'delta mfcc_10', 'delta mfcc_11', 'delta mfcc_12', 'delta mfcc_13', 'delta chroma_1', 'delta chroma_2', 'delta chroma_3', 'delta chroma_4', 'delta chroma_5', 'delta chroma_6', 'delta chroma_7', 'delta chroma_8', 'delta chroma_9', 'delta chroma_10', 'delta chroma_11', 'delta chroma_12', 'delta chroma_std']
df.head()

# **My Voice Analysis**

trial  https://github.com/Shahabks/my-voice-analysis  <br?

note- Put the audio file at the same location where your myspsolution.praat file is or vice-versa

In [ ]:
!git clone https://github.com/Shahabks/my-voice-analysis  

Cloning into 'my-voice-analysis'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 158 (delta 0), reused 0 (delta 0), pack-reused 155
Receiving objects: 100% (158/158), 479.18 KiB | 3.90 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [ ]:
 pip install my-voice-analysis

     |████████████████████████████████| 9.0MB 4.3MB/s 


In [ ]:
pip install -U my-voice-analysis

Requirement already up-to-date: my-voice-analysis in /usr/local/lib/python3.6/dist-packages (0.7)


In [ ]:
mysp =__import__("my-voice-analysis")

p="Aaditya Dinesh Thokade" # Audio File title
c=r"/content/gdrive/My Drive/EvueMe New Dataset/Audio/" # Path to the Audio_File directory (Python 3.7)
a = mysp.myspgend(p,c)      

Try again the sound of the audio was not clear


In [ ]:
mysp =__import__("my-voice-analysis")

p="Aavriti Arora" # Audio File title
c=r"/content/" # Path to the Audio_File directory (Python 3.7)
a = mysp.myspgend(p,c)      

[]
Voice not recognized


In [ ]:
mysp.mysppron(p,c)

[]
Pronunciation_posteriori_probability_score_percentage= :84.91


In [ ]:
mysp.myspsyl(p,c)          

[]
number_ of_syllables= 171


In [ ]:
mysp.mysppaus(p,c)

[]
number_of_pauses= 22


In [ ]:
mysp.myspsr(p,c)            

[]
rate_of_speech= 3 # syllables/sec original duration


In [ ]:
mysp.myspatc(p,c)

[]
articulation_rate= 5 # syllables/sec speaking duration


In [ ]:
mysp.myspst(p,c)

[]
speaking_duration= 34.5 # sec only speaking duration without pauses


In [ ]:
mysp.myspod(p,c)

[]
original_duration= 53.7 # sec total speaking duration with pauses


In [ ]:
mysp.myspbala(p,c)

[]
balance= 0.6 # ratio (speaking duration)/(original duration)


In [ ]:
mysp.myspf0mean(p,c)

[]
f0_mean= 155.77 # Hz global mean of fundamental frequency distribution


In [ ]:
mysp.myspf0sd(p,c)

[]
f0_SD= 15.18 # Hz global standard deviation of fundamental frequency distribution


In [ ]:
mysp.myspf0med(p,c)

[]
f0_MD= 153.6 # Hz global median of fundamental frequency distribution


In [ ]:
mysp.myspf0min(p,c)

[]
f0_min= 98 # Hz global minimum of fundamental frequency distribution


In [ ]:
mysp.myspf0max(p,c)

[]
f0_max= 231 # Hz global maximum of fundamental frequency distribution


In [ ]:
mysp.myspf0q25(p,c)

[]
f0_quan25= 146 # Hz global 25th quantile of fundamental frequency distribution


In [ ]:
mysp.myspf0q75(p,c)

[]
f0_quan75= 165 # Hz global 75th quantile of fundamental frequency distribution


In [ ]:
mysp.mysptotal(p,c)

[]
                           0
number_ of_syllables     765
number_of_pauses          84
rate_of_speech             4
articulation_rate          5
speaking_duration      142.3
original_duration      206.6
balance                  0.7
f0_mean               252.22
f0_std                 40.77
f0_median              246.4
f0_min                    77
f0_max                   421
f0_quantile25            229
f0_quan75                270


#**Myprosody**

Note- clone the github repository of myprosody and then upload your own myprosody.py file in the myprosody/myprosody/dataset/audioFiles/ of the github repository along with the audio files there and then import it along with pickle also

In [ ]:
pwd

'/content'

In [ ]:
!git clone https://github.com/Shahabks/myprosody
!pip install -r myprosody/requirements.txt

Cloning into 'myprosody'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 167
Receiving objects: 100% (171/171), 2.63 MiB | 2.48 MiB/s, done.
Resolving deltas: 100% (78/78), done.
     |████████████████████████████████| 9.0MB 2.5MB/s 


In [ ]:
!pip install myprosody

In [ ]:
pwd

'/content'

In [ ]:
cd /content/myprosody/myprosody/dataset/audioFiles

/content/myprosody/myprosody/dataset/audioFiles


In [ ]:
import pickle

In [ ]:
import myprosody as mysp

In [ ]:
p="Yatharth agrawal" # Audio File title
c=r"/content/myprosody/myprosody/" # Path to the Audio_File directory (Python 3.7)

In [ ]:
mysp.mysptotal(p,c)

[]
                           0
number_ of_syllables     489
number_of_pauses          72
rate_of_speech             3
articulation_rate          5
speaking_duration      105.7
original_duration      150.3
balance                  0.7
f0_mean               161.36
f0_std                  34.8
f0_median              154.5
f0_min                    71
f0_max                   401
f0_quantile25            143
f0_quan75                169


In [ ]:
mysp.myspgend(p,c)

[]
a female, mood of speech: speaking passionately, p-value/sample size= :0.00 5


In [ ]:
mysp.myspsyl(p,c)

[]
number_ of_syllables= 171


171

In [ ]:
mysp.mysppaus(p,c)

[]
number_of_pauses= 22


22

In [ ]:
mysp.myspsr(p,c)

[]
rate_of_speech= 3 # syllables/sec original duration


3

In [ ]:
mysp.myspatc(p,c)

[]
articulation_rate= 5 # syllables/sec speaking duration


5

In [ ]:
mysp.myspst(p,c)

[]
speaking_duration= 34.5 # sec only speaking duration without pauses


34.5

In [ ]:
mysp.myspod(p,c)

[]
original_duration= 53.7 # sec total speaking duration with pauses


53.7

In [ ]:
mysp.myspbala(p,c)

[]
balance= 0.6 # ratio (speaking duration)/(original duration)


0.6

In [ ]:
mysp.myspf0mean(p,c)

[]
f0_mean= 155.77 # Hz global mean of fundamental frequency distribution


155.77

In [ ]:
mysp.myspf0sd(p,c)

[]
f0_SD= 15.18 # Hz global standard deviation of fundamental frequency distribution


15.18

In [ ]:
mysp.myspf0med(p,c)

[]
f0_MD= 153.6 # Hz global median of fundamental frequency distribution


153.6

In [ ]:
mysp.myspf0min(p,c)

[]
f0_min= 98 # Hz global minimum of fundamental frequency distribution


98

In [ ]:
mysp.myspf0max(p,c)

[]
f0_max= 231 # Hz global maximum of fundamental frequency distribution


231

In [ ]:
mysp.myspf0q25(p,c)

[]
f0_quan25= 146 # Hz global 25th quantile of fundamental frequency distribution


146

In [ ]:
mysp.myspf0q75(p,c)

[]
f0_quan75= 165 # Hz global 75th quantile of fundamental frequency distribution


165

In [ ]:
mysp.mysppron(p,c)

[]
Pronunciation_posteriori_probability_score_percentage= :95.02


In [ ]:
mysp.myprosody(p,c)

Compared to native speech, here are the prosodic features of your speech:
average_syll_pause_duration:	 25.000000 (% percentile )
No._long_pause:	 (:Out of Range)
speaking_time:	 (:Out of Range)
ave_No._of_words_in_minutes:	 (:Out of Range)
articulation_rate:	 66.666667 (% percentile )
No._words_in_minutes:	 (:Out of Range)
formants_index:	 25.000000 (% percentile )
f0_index:	 (:Out of Range)
f0_quantile_25_index:	 (:Out of Range)
f0_quantile_50_index:	 (:Out of Range)
f0_quantile_75_index:	 (:Out of Range)
f0_std:	 66.666667 (% percentile )
f0_max:	 66.666667 (% percentile )
f0_min:	 33.333333 (% percentile )
No._detected_vowel:	 (:Out of Range)
perc%._correct_vowel:	 (:Out of Range)
(f2/f1)_mean:	 66.666667 (% percentile )
(f2/f1)_std:	 66.666667 (% percentile )
no._of_words:	 (:Out of Range)
no._of_pauses:	 (:Out of Range)
intonation_index:	 25.000000 (% percentile )
(voiced_syll_count)/(no_of_pause):	 66.666667 (% percentile )
TOEFL_Scale_Score:	 66.666667 (% percentile )
Score_Sha

In [ ]:
mysp.mysplev(p,c)

[]
Try again the sound of the audio was not clear


## Myprosody code for multiple audio files on local disk and run on anaconda

**Note**  
<br>
1) Names of the audio files must not contain any ***dot*** <br>
<br>
2) ***Procedure to run it on local disk (not Colab)*** Download the https://github.com/Shahabks/myprosody zip file on Desktop and then after extracting Audio File names, keep the myprosody.py file in C:\\Users\\Avinash_PC as well as in the file location C:\Users\Avinash_PC\Desktop\myprosody\myprosody-master\myprosody\dataset\audioFiles along with the audio files which need to be processed. <br>

However it is not necessary to keep myprosody.py file in the audioFiles folder where you keep our audio files


In [ ]:
pwd

In [ ]:
cd C:/Users/Avinash_PC/Desktop/myprosody/myprosody-master/myprosody/dataset/audioFiles

In [ ]:
import os

!pip install myprosody

import pickle
import myprosody as mysp

import numpy as np
import pandas as pd

**Audio File Name Extraction** <br>
Make sure that you only have audio files (.wav) and not any other kind of file befoe going for Name Extraction

In [ ]:
NameList = []

filepath = "C:/Users/Avinash_PC/Desktop/myprosody/myprosody-master/myprosody/dataset/audioFiles/"
for audio in os.listdir(filepath):
    NameList.append(audio.split('.')[0])

NameList, len(NameList), type(NameList)

In [ ]:
myprosody_list = []

for i in range(0,len(NameList)):
  p = NameList[i]
  c = c=r"C:/Users/Avinash_PC/Desktop/myprosody/myprosody-master/myprosody/"
  myprosody_df = mysp.mysptotal(p,c)
  myprosody_list.append(myprosody_df)
  print(i)

In [ ]:
myprosody_features_df = myprosody_list[0].T

for i in range(1,len(NameList)):
    myprosody_features_df = myprosody_features_df.append(myprosody_list[i].T)

# Using DataFrame.insert() to add a column 
myprosody_features_df.insert(0, "Name", NameList, True) 

myprosody_features_df

In [ ]:
myprosody_features_df.to_csv("C:/Users/Avinash_PC/Desktop/Myprosody Features.csv")